<a href="https://colab.research.google.com/github/Arshad221b/Automatic-Music-Transcription/blob/master/AMT_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import keras
from keras.layers import Dense, Flatten, Reshape, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from  skimage.measure import block_reduce

from PIL import Image
import pretty_midi
import os, os.path
import sys

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path_image = '/content/drive/My Drive/DATA2/data4'
path_midi  = '/content/drive/My Drive/DATA2/data2'

In [0]:
class util():
  def pretty_midi_to_one_hot(pm, fs=100):

      one_hots = []

      if len(pm.instruments) < 1:
          return 0

      for instrument in pm.instruments:
          one_hot = np.zeros((128, int(fs*instrument.get_end_time())+1))
          for note in instrument.notes:
              # note on
              one_hot[note.pitch, int(note.start*fs)] = 1
              # print('note on',note.pitch, int(note.start*fs))
              # note off
              one_hot[note.pitch, int(note.end*fs)] = 0
              # print('note off',note.pitch, int(note.end*fs))
          one_hots.append(one_hot)

      one_hot = np.zeros((128, np.max([o.shape[1] for o in one_hots])))
      for o in one_hots:
          one_hot[:, :o.shape[1]] += o

      one_hot = np.clip(one_hot,-1,1)
      return one_hot



  def one_hot_to_pretty_midi(one_hot, fs=100, program=1,bpm=120):

      notes, frames = one_hot.shape
      pm = pretty_midi.PrettyMIDI()
      instrument = pretty_midi.Instrument(program=program)

      # prepend, append zeros so we can acknowledge inital and ending events
      piano_roll = np.hstack((np.zeros((notes, 1)),
                              one_hot,
                              np.zeros((notes, 1))))

      # use changes to find note on / note off events
      changes = np.nonzero(np.diff(piano_roll).T)

      # keep track of note on times and notes currently playing
      note_on_time = np.zeros(notes)
      current_notes = np.zeros(notes)

      bps = bpm / 60
      beat_interval = fs / bps
      strong_beats = beat_interval * 2 #(for 4/4 timing)

      last_beat_time = 0

      for time, note in zip(*changes):
          change = piano_roll[note, time + 1]

          if time >= last_beat_time + beat_interval:
              for note in current_notes:
                  time = time / fs

          time = time / fs
          if change == 1:
              # note on
              if current_notes[note] == 0:
                  # from note off
                  note_on_time[note] = time
                  current_notes[note] = 1


          elif change == 0:
              #note off
              pm_note = pretty_midi.Note(
                      velocity=100, #don't care fer now
                      pitch=note,
                      start=note_on_time[note],
                      end=time)
              current_notes[note] = 0
              instrument.notes.append(pm_note)
      pm.instruments.append(instrument)
      return pm

  def slice_to_categories(piano_roll):
      notes_list = np.zeros(128)
      notes = np.nonzero(piano_roll)[0]
      notes = np.unique(notes)

      for note in notes:
          notes_list[note] = 1

      return notes_list

In [0]:
import numpy as np
import keras
from keras.layers import Dense, Flatten, Reshape, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from  skimage.measure import block_reduce
from keras.models import Model
from PIL import Image
import pretty_midi
import os, os.path
import sys
def pretty_midi_to_one_hot(pm, fs=100):

    # Allocate a matrix of zeros - we will add in as we go
    one_hots = []

    if len(pm.instruments) < 1:
        return 0

    for instrument in pm.instruments:
        one_hot = np.zeros((128, int(fs*instrument.get_end_time())+1))
        for note in instrument.notes:
            # note on
            one_hot[note.pitch, int(note.start*fs)] = 1
            # print('note on',note.pitch, int(note.start*fs))
            # note off
            one_hot[note.pitch, int(note.end*fs)] = 0
            # print('note off',note.pitch, int(note.end*fs))
        one_hots.append(one_hot)

    one_hot = np.zeros((128, np.max([o.shape[1] for o in one_hots])))
    for o in one_hots:
        one_hot[:, :o.shape[1]] += o

    one_hot = np.clip(one_hot,-1,1)
    return one_hot

def slice_to_categories(piano_roll):
    notes_list = np.zeros(128)
    notes = np.nonzero(piano_roll)[0]
    notes = np.unique(notes)

    for note in notes:
        notes_list[note] = 1

    return notes_list

In [0]:
import pandas as pd
def train(x_train, y_train, x_test, y_test):
    batch_size = 16
    num_classes = 128
    epochs = 50

    # input image dimensions
    img_x, img_y = 308, 601

    path = '/content/drive/My Drive/MainDATA/'
    #path = '/mnt/c/Users/chau/Documents/models'
    model_ckpt = os.path.join(path,'model_SPeech_3.h5')
    
    #x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 3)
    #x_test  = x_test.reshape(x_train.shape[0], img_x, img_y, 3)
    
    # Convert data to right type
    #x_train = x_train.astype('float32')
    #x_test = x_test.astype('float32')
    #x_train /= 255
    #x_test /= 255
    #print(x_train)
    #print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    model = create_model2()
    model.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.SGD(lr=0.01),
            metrics=['accuracy'])

    history = AccuracyHistory()

    checkpoint = ModelCheckpoint(model_ckpt,
            monitor='val_loss',
            verbose=1,
            save_best_only=True,
            mode='min')
    early_stop = EarlyStopping(patience=5, 
            monitor='val_loss',
            verbose=1, mode='min')
    callbacks = [history, checkpoint,early_stop]
    
    model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test),
            callbacks=callbacks)

    score = model.evaluate(x_test, y_test, batch_size = 1)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    #plt.plot(range(1,101), history.acc)
    #plt.xlabel('Epochs')
    #plt.ylabel('Accuracy')
    #plt.savefig('loss.png')
    #plt.show()

In [0]:
test_path_image = '/content/drive/My Drive/MainDATA/test_spec'
test_path_midi  = '/content/drive/My Drive/MainDATA/test_split'

In [0]:
from keras.applications import VGG19
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import model_from_json
from keras.optimizers import Nadam
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import MaxPooling2D

def create_model2():
  img_x, img_y = 308, 601
  input_shape = (img_x, img_y, 3)
  num_classes = 128


  base_model = Xception(input_shape=input_shape, weights='imagenet', include_top=False)
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  predictions = Dense(num_classes, activation='softmax')(x)
  model = Model(base_model.input, predictions)
  return model

In [0]:
def create_model():
    img_x, img_y = 308, 601
    input_shape = (img_x, img_y, 3)
    num_classes = 128

    model = Sequential()
    model.add(LSTM(input_shape = input_shape,return_sequences=True,stateful=False ))
    
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

  
    
    #model.add(Conv2D(64, (5,5), activation='relu'))
    # Final output layer
    #model.add(Conv2D(128, (5,5), activation='sigmoid'))
    #model.add(Flatten())
    model.add(Flatten())
    #model.add(Dense(64, activation='sigmoid'))

    model.add(Dense(num_classes, activation='softmax'))
    return model
    



class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

In [0]:
x_train, y_train = [], []
x_test, y_test = [], []

In [13]:
path_image = '/content/drive/My Drive/DATA2/data4'
path_midi  = '/content/drive/My Drive/DATA2/data2'
def run_cnn(image_path, midi_path,test_path_image, test_path_midi):
    # x is spectrogram, y is MIDI
    #jpg_path = '/mnt/d/Workspace/EE379K/data/spectrograms'
    #midi_path = '/mnt/d/Workspace/EE379K/data/split_midi'
    #jpg_path = '/mnt/c/Users/chau/Documents/spectrograms'
    #midi_path = '/mnt/c/Users/chau/Documents/split_midi'
    x_train, y_train = [], []
    x_test, y_test = [], []

    img = []
    i = 0
    for filename in os.listdir(image_path):
        print(filename +"\t" + str(i))
        # filename = "daylight_128.jpg"
        m_fn = filename.replace(".png", ".mid")
        if os.path.isfile(os.path.join(midi_path, m_fn)):
            pm = pretty_midi.PrettyMIDI(os.path.join(midi_path, m_fn))
            oh = pretty_midi_to_one_hot(pm)
            if type(oh) is not int:
                oh = slice_to_categories(oh)
                #oh = oh.reshape(1, 128)
                y_train.append(oh)
        
                im = Image.open(os.path.join(image_path, filename)) 
                #im = im.crop((14, 13, 594, 301))
                #resize = im.resize((49, 145), Image.NEAREST)
                #resize.load()
                #result = Image.fromarray((visual * 255).astype(numpy.uint8))
                #resize.save("images/" + str(i) + ".jpg")
                im = im.convert("RGB")
                arr = np.asarray(im, dtype="uint8")
                #print(arr)
                #arr = block_reduce(arr, block_size=(2,2,1), func=np.mean)
                
                x_train.append(arr)
                #if len(x_train) > 0:
                #    break
                i += 1
    print("================================================================================================================================")
    
    for filename in os.listdir(test_path_image):
        print(filename)
        # filename = "daylight_128.jpg"
        m_fn = filename.replace(".jpg", ".mid")
        if os.path.isfile(os.path.join(test_path_midi, m_fn)):
            pm = pretty_midi.PrettyMIDI(os.path.join(test_path_midi, m_fn))
            oh = pretty_midi_to_one_hot(pm)
            if type(oh) is not int:
                oh = slice_to_categories(oh)
                #oh = oh.reshape(1, 128)
                y_test.append(oh)
        
                im = Image.open(os.path.join(test_path_image, filename)) 
                #im = im.crop((14, 13, 594, 301))
                resize = im.resize((601, 308), Image.NEAREST)
                #resize.load()
                #result = Image.fromarray((visual * 255).astype(numpy.uint8))
                #resize.save("images/" + str(i) + ".jpg")
                arr = np.asarray(resize, dtype="uint8")
                #print(arr)
                #arr = block_reduce(arr, block_size=(2,2,1), func=np.mean)
                x_test.append(arr)
                #if len(x_train) > 0:
                #    break
                i += 1


    x_train = np.array(x_train)
    #x_train = x_train.reshape(len(x_train), 1)
    y_train = np.array(y_train)
    x_test  = np.array(x_test)
    y_test  = np.array(y_test)
    #print(y_train)
    #print(x_train.shape)
    #print(y_train.shape)
    #print(len(x_train))
    #print(np.shape(x_train))
    #im_array = np.array([np.array
    #x_train = np.array(x_train)
    #x_test = np.copy(x_train)
    #y_test = np.copy(y_train)
    #x_train, x_test, y_train, y_test = train_test_split(
    #        x_train, y_train, test_size=0.2, random_state=1)
    #print(x_train.shape)
    #print(y_train.shape)
    #x_train /= 255.0
    #x_test /= 255.0
    train(x_train, y_train, x_test, y_test)

run_cnn(path_image, path_midi, test_path_image, test_path_midi)

song1_819.png	0
song3_539.png	0
song13_97.png	0
song4_43.png	0
song6_468.png	1
song5_1166.png	2
song4_960.png	2
song1_831.png	2
song14_1.png	2
song13_83.png	2
song4_975.png	3
song12_199.png	4
song7_109.png	5
song4_57.png	5
song4_974.png	6
song5_1172.png	7
song10_249.png	7
song5_1173.png	8
song8_588.png	8
song1_825.png	9
song10_248.png	9
song12_198.png	10
song4_42.png	11
song8_589.png	12
song13_82.png	13
song1_824.png	13
song4_56.png	13
song14_0.png	13
song13_96.png	14
song3_538.png	15
song1_830.png	15
song7_108.png	15
song4_961.png	15
song5_1167.png	15
song10_274.png	15
song6_469.png	15
song3_276.png	16
song4_949.png	16
song5_582.png	16
song6_441.png	16
song6_327.png	17
song4_791.png	18
song11_701.png	19
song1_818.png	19
song7_134.png	19
song14_136.png	20
song2_703.png	21
song3_510.png	21
song2_717.png	22
song3_77.png	22
song3_63.png	22
song7_120.png	22
song6_333.png	23
song14_122.png	24
song13_348.png	25
song3_504.png	25
song5_596.png	25
song10_260.png	25
song6_455.png	26
song3_262.pn

In [0]:
z='/content/drive/My Drive/MainDATA/data4/song3_300.jpg'


In [0]:
from keras.models import load_model
model = load_model('/content/drive/My Drive/MainDATA/ckpt3.h5')
im = Image.open(z)
x  = []
arr = np.asarray(im, dtype="uint8")
x.append(arr)
x = np.array(x)
#x /= 255.0
y1 = model.predict(x)

In [0]:
import numpy as np
result = np.where(y1 == np.amax(y1))

In [0]:
print(result)